In [23]:
from IPython.display import Markdown, display
from openai import OpenAI
import json
import os

In [24]:
client = OpenAI(
  api_key=os.environ['OPENAI_API_KEY'],  # this is also the default, it can be omitted
)

In [25]:
model_used = "gpt-4o-mini-2024-07-18"
#"gpt-4o-mini-2024-07-18"

In [26]:
# Create the system_prompt
example_simple = """User input: "חזון ישעיהו בןאמוץ אשר חזה עליהודה וירושלם בימי עזיהו יותם אחז יחזקיהו מלכי יהודה"
Expected output: ["חזון ישעיהו בןאמוץ","אשר חזה עליהודה וירושלם בימי עזיהו יותם אחז יחזקיהו מלכי יהודה"]"""

example_complex = """User input: "יביא יהוה עליך ועלעמך ועלבית אביך ימים אשר לאבאו למיום סוראפרים מעל יהודה את מלך אשור פ"
Expected output: ["יביא יהוה עליך ועלעמך ועלבית אביך ימים את מלך אשור פ" ,"אשר לאבאו למיום" ,"סוראפרים מעל יהודה"]"""


SYSTEM_PROMPT = f"Identify clauses in this Biblical Hebrew verse and return a JSON list containing the clauses, as shown in the following examples. Do not write anything else than the JSON list in your output. Example 1 {example_simple} Example 2 {example_complex}"

In [33]:
# Functions

def parse_verse (verse):
    """Send a request to the chosen OpenAI model to parse a Biblical Hebrew verse into clauses."""
    try:
        response = client.chat.completions.create(
            model=model_used,
            messages=[
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": f"{verse}"}
            ]
        )
        # Extract and return the response
        return response.choices[0].message
    except Exception as e:
        print(f"Error processing verse: {verse}")
        print(e)
        return None


def process_file(input_file, output_file):
    """Read a JSONL file containing verses, process them and save results to another JSONL file."""
    with open(input_file, "r", encoding="utf-8") as infile, open(output_file, "w", encoding="utf_8") as outfile:
        for line in infile:
            data=json.loads(line)
            verse = data["verse"]
            result = parse_verse(verse).content
            if result:
                output = {"verse": verse, "parsed_clauses": result}
                outfile.write(json.dumps(output, ensure_ascii=False) + "\n")

# Input and output files
input_jsonl_file = "data/all_verses_dataset.jsonl"
output_jsonl_file = "data/all_parsed_clauses_gpt4o-mini.jsonl"

In [29]:
# Process test file
result = parse_verse("Parse this verse: ויהישם עםיהוה ארבעים יום וארבעים לילה לחם לא אכל ומים לא שתה ויכתב עלהלחת את דברי הברית עשרת הדברים")

In [30]:
print(result)

ChatCompletionMessage(content='["ויהישם עםיהוה ארבעים יום וארבעים לילה","לחם לא אכל ומים לא שתה","ויקתב עלהלחת את דברי הברית עשרת הדברים"]', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)


In [31]:
result.content

'["ויהישם עםיהוה ארבעים יום וארבעים לילה","לחם לא אכל ומים לא שתה","ויקתב עלהלחת את דברי הברית עשרת הדברים"]'

In [ ]:
process_file(input_jsonl_file, output_jsonl_file)